In [1]:
import os
import pickle

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from src.data_preparation.data_preprocessing import DataReshaperLSTM
from src.visualization.charts import ModelCharts
from src.visualization.metrics import PredictionEvaluator
from src.config import Config

## Load the data

In [2]:
config = Config()
variant_co2 = 'co2'

#lstm
train_lstm_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/train.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])
test_lstm_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/test.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])

with open(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/data_preprocessor_lstm.pkl'), 'rb') as f:
    data_preprocessor_lstm = pickle.load(f)

#residual lightgbm
train_lightgbm_residuals_df = pd.read_csv(os.path.join(config.output_cleaned_hybrid, f'lightgbm_lstm/{variant_co2}/residuals_train.csv')).set_index(["year", "country"]).rename_axis(index={"country": config.additional_index})
test_lightgbm_residuals_df = pd.read_csv(os.path.join(config.output_cleaned_hybrid, f'lightgbm_lstm/{variant_co2}/residuals_test.csv')).set_index(["year", "country"]).rename_axis(index={"country": config.additional_index})
train_lightgbm_residuals_preprocessed, test_lightgbm_residuals_preprocessed = data_preprocessor_lstm.preprocess_data(train_lightgbm_residuals_df, test_lightgbm_residuals_df)

In [3]:
train_lstm_df

,,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1853,Stany Zjednoczone,34,0.016985,0.010509,0.000000,0.119705,0.000000,0.005568,0.000000,0.000000,0.449621,...,0.010347,0.006536,0.115255,0.000000,0.005462,0.000000,0.000000,0.351582,0.000000,0.142663
1854,Stany Zjednoczone,34,0.017544,0.011437,0.006623,0.125772,0.000000,0.006055,0.000000,0.000000,0.473915,...,0.011483,0.006536,0.116109,0.000000,0.006149,0.000000,0.000000,0.355254,0.000000,0.148196
1855,Stany Zjednoczone,34,0.018120,0.012690,0.006623,0.126702,0.000000,0.006817,0.000000,0.000000,0.478863,...,0.011840,0.013072,0.120435,0.000000,0.006760,0.000000,0.000000,0.368256,0.000000,0.150416
1856,Stany Zjednoczone,34,0.018712,0.013083,0.013245,0.131413,0.000000,0.007495,0.000000,0.000000,0.496390,...,0.011940,0.013072,0.122052,0.000000,0.007779,0.000000,0.000000,0.372171,0.000000,0.154770
1857,Stany Zjednoczone,34,0.019322,0.013194,0.013245,0.133173,0.000000,0.008625,0.000000,0.000000,0.501668,...,0.012561,0.013072,0.125706,0.000000,0.008162,0.000000,0.000000,0.382568,0.000000,0.156695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,Szwecja,35,0.005112,0.027868,0.013245,0.008810,0.002694,0.002540,0.003924,0.001725,0.046782,...,0.026320,0.013072,0.008216,0.002757,0.002288,0.004482,0.001754,0.036047,0.017440,0.021023
1997,Szwecja,35,0.005147,0.029056,0.013245,0.009214,0.003148,0.002537,0.004482,0.001754,0.048590,...,0.026822,0.013072,0.008650,0.002606,0.002525,0.004283,0.001892,0.035421,0.018473,0.020478
1998,Szwecja,35,0.005158,0.029609,0.013245,0.009687,0.002976,0.002800,0.004283,0.001892,0.047746,...,0.027630,0.013072,0.008228,0.002351,0.002198,0.004762,0.001918,0.035754,0.017144,0.020634


In [4]:
train_lightgbm_residuals_preprocessed

,,residual
year,country_index,
1961,Arabia Saudyjska,0.244953
1962,Arabia Saudyjska,0.245599
1963,Arabia Saudyjska,0.245623
1964,Arabia Saudyjska,0.245102
1965,Arabia Saudyjska,0.244237
...,...,...
2000,Włochy,0.244847
2001,Włochy,0.243480
2002,Włochy,0.243247


In [5]:
#combine train df
train_hybrid_df = pd.merge(train_lightgbm_residuals_preprocessed, train_lstm_df, left_index=True, right_index=True)
train_hybrid_df

,,residual,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1961,Arabia Saudyjska,0.244953,0,0.000194,0.002745,0.000000,0.000000,0.000087,0.000000,0.000000,0.000000,...,0.002734,0.000000,0.000000,0.000094,0.000000,0.000000,0.000000,0.038131,0.001042,0.014038
1962,Arabia Saudyjska,0.245599,0,0.000261,0.003040,0.000000,0.000267,0.000107,0.000000,0.000000,0.000000,...,0.003084,0.000000,0.000377,0.000108,0.000000,0.000000,0.000000,0.038220,0.001393,0.014426
1963,Arabia Saudyjska,0.245623,0,0.000331,0.003426,0.000000,0.000678,0.000124,0.000000,0.000000,0.000000,...,0.003477,0.000000,0.001322,0.000194,0.000000,0.000000,0.000019,0.038285,0.002431,0.014563
1964,Arabia Saudyjska,0.245102,0,0.000403,0.003860,0.000000,0.001707,0.000221,0.000000,0.000000,0.000019,...,0.003799,0.000000,0.001592,0.000208,0.000000,0.000000,0.000025,0.038396,0.002698,0.014572
1965,Arabia Saudyjska,0.244237,0,0.000478,0.004214,0.000000,0.002001,0.000238,0.000000,0.000000,0.000025,...,0.004152,0.000000,0.001634,0.000270,0.000000,0.000000,0.001053,0.038383,0.002248,0.014198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000,Włochy,0.244847,44,0.042144,0.202657,0.046358,0.069297,0.035316,0.012401,0.051699,0.100876,...,0.187879,0.045752,0.063157,0.032498,0.010996,0.041351,0.109807,0.027605,0.102774,0.072296
2001,Włochy,0.243480,44,0.042163,0.207264,0.046358,0.069041,0.037112,0.012192,0.041351,0.109807,...,0.196229,0.052288,0.063426,0.033743,0.011423,0.043386,0.114710,0.026136,0.100887,0.071542
2002,Włochy,0.243247,44,0.042187,0.216475,0.052980,0.069334,0.038533,0.012664,0.043386,0.114710,...,0.201132,0.052288,0.062427,0.034874,0.012033,0.040846,0.114960,0.024196,0.099381,0.071573


In [6]:
#combine test df
test_hybrid_df = pd.merge(test_lightgbm_residuals_preprocessed, test_lstm_df, left_index=True, right_index=True)
test_hybrid_df

,,residual,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1989,Stany Zjednoczone,0.378038,34,0.186605,0.980192,1.013245,0.770779,0.080107,0.386410,0.088209,0.785603,...,0.925972,1.013072,0.735932,0.070727,0.365490,0.101656,0.820841,0.131566,0.887696,0.744042
1990,Stany Zjednoczone,0.379906,34,0.188312,1.021423,1.026490,0.801646,0.080768,0.405226,0.101656,0.820841,...,0.958024,1.026144,0.745954,0.070761,0.369653,0.101017,0.874110,0.130467,0.887150,0.744836
1991,Stany Zjednoczone,0.343014,34,0.190056,1.056778,1.039735,0.812559,0.080807,0.409841,0.101017,0.874110,...,0.974791,1.045752,0.743029,0.071229,0.369909,0.616333,0.870813,0.136453,0.862972,0.729761
1992,Stany Zjednoczone,0.204248,34,0.192119,1.075274,1.059603,0.809374,0.081341,0.410125,0.616333,0.870813,...,0.972084,1.058824,0.734748,0.069638,0.366365,0.603152,0.886989,0.118116,0.840136,0.738090
1993,Stany Zjednoczone,0.303004,34,0.194488,1.072287,1.072848,0.800357,0.079524,0.406197,0.603152,0.886989,...,1.006652,1.071895,0.747962,0.070185,0.370468,0.598723,0.919292,0.101080,0.861291,0.749304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Szwecja,0.242516,35,0.006089,0.052010,0.019868,0.009155,0.003605,0.001879,0.010654,0.001542,...,0.048052,0.019608,0.007950,0.003419,0.001650,0.010973,0.001605,0.039618,0.011526,0.019703
2020,Szwecja,0.241296,35,0.006179,0.053028,0.019868,0.008925,0.003904,0.001829,0.010973,0.001605,...,0.049011,0.019608,0.007811,0.002887,0.001654,0.007129,0.001515,0.039806,0.011298,0.019193
2021,Szwecja,0.242602,35,0.006258,0.054085,0.019868,0.008773,0.003296,0.001834,0.007129,0.001515,...,0.047943,0.019608,0.007342,0.002706,0.001355,0.005919,0.001227,0.039930,0.010443,0.019574


## Reshape the data

In [7]:
data_resherper = DataReshaperLSTM()
x_train, x_test, y_train, y_test = data_resherper.reshape_data(train_hybrid_df, test_hybrid_df)

## Build the model

In [8]:
input_shape = (x_train.shape[1], x_train.shape[2])
output_units = 1

model = Sequential([
    LSTM(50, input_shape=input_shape, return_sequences = True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(output_units, activation='relu')
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             14800     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 35,051
Trainable params: 35,051
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [9]:
history = model.fit(x_train, y_train, epochs=config.epochs, batch_size=config.batch_size)

Epoch 1/15
3153/3153 [==============================] - 3s 703us/step - loss: 0.0038
Epoch 2/15
3153/3153 [==============================] - 2s 701us/step - loss: 0.0013
Epoch 3/15
3153/3153 [==============================] - 2s 702us/step - loss: 0.0011
Epoch 4/15
3153/3153 [==============================] - 3s 911us/step - loss: 9.9335e-04
Epoch 5/15
3153/3153 [==============================] - 3s 800us/step - loss: 9.1253e-04
Epoch 6/15
3153/3153 [==============================] - 2s 761us/step - loss: 9.4131e-04
Epoch 7/15
3153/3153 [==============================] - 2s 748us/step - loss: 8.2187e-04
Epoch 8/15
3153/3153 [==============================] - 2s 720us/step - loss: 7.5504e-04
Epoch 9/15
3153/3153 [==============================] - 2s 717us/step - loss: 7.1382e-04
Epoch 10/15
3153/3153 [==============================] - 2s 743us/step - loss: 6.6237e-04
Epoch 11/15
3153/3153 [==============================] - 2s 766us/step - loss: 6.5226e-04
Epoch 12/15
3153/3153 [========

In [10]:
# Save the model
model.save(os.path.join(config.models_folder, f'{variant_co2}_lightgbm_lstm_model.h5'))

# Evaluate the model on the test set
loss = model.evaluate(x_test, y_test, verbose = 0)

## Predictions

In [11]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

inverted_data_predicted_train_y = data_preprocessor_lstm.inverse_transform_data(train_predictions, train_predictions.shape[0], train_hybrid_df.shape[1]-2)
inverted_data_train_y = data_preprocessor_lstm.inverse_transform_data(y_train, train_predictions.shape[0], train_hybrid_df.shape[1]-2)

inverted_data_predicted_test_y = data_preprocessor_lstm.inverse_transform_data(test_predictions, test_predictions.shape[0], test_hybrid_df.shape[1]-2)
inverted_data_test_y = data_preprocessor_lstm.inverse_transform_data(y_test, test_predictions.shape[0], test_hybrid_df.shape[1]-2)

26/26 [==============================] - 0s 529us/step


In [12]:
train_df_reset = train_hybrid_df.reset_index()
test_df_reset = test_hybrid_df.reset_index()

co2_predicted_train = inverted_data_predicted_train_y[:, -1]
co2_actual_train = inverted_data_train_y[:, -1]
co2_predicted_test = inverted_data_predicted_test_y[:, -1]
co2_actual_test = inverted_data_test_y[:, -1]

# Create DataFrames for train and test
train_results = pd.DataFrame({
    "country": train_df_reset["country_index"].values,
    "year": train_df_reset["year"].values,
    "co2_predicted": co2_predicted_train,
    "co2_actual": co2_actual_train
})

test_results = pd.DataFrame({
    "country": test_df_reset["country_index"].values,
    "year": test_df_reset["year"].values,
    "co2_predicted": co2_predicted_test,
    "co2_actual": co2_actual_test
})


train_results.to_csv(os.path.join(config.predictions_hybrid_lightgbm_lstm, f'lightgbm_lstm_{variant_co2}_train.csv'))
test_results.to_csv(os.path.join(config.predictions_hybrid_lightgbm_lstm, f'lightgbm_lstm_{variant_co2}_test.csv'))

## Charts

In [13]:
charts = ModelCharts(train_results, test_results)

#Line plot
charts.generate_line_plot(variant = variant_co2, model_output_file=config.predictions_hybrid_lightgbm_lstm)
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_lightgbm_lstm, dataset_type='train')
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_lightgbm_lstm, dataset_type='test')

#Scatter plot
charts.generate_scatter_plot(variant = variant_co2, model_output_file=config.predictions_hybrid_lightgbm_lstm)
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_lightgbm_lstm, dataset_type='train')
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_lightgbm_lstm, dataset_type='test')

## Metrics

In [14]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_results, test_results, actual_col='co2_actual', predicted_col='co2_predicted', variant = variant_co2, model_output_file=config.metrics_hybrid)